# 学习笔记
严格来说，这个是在学习《实战Google深度学习框架》时记录并实践一些局部的知识点或者一些小技巧，所以可能不同部分之间没有明显联系，都是独立从章节中抽取出来。相对完整的章节内容，都按照章节或者模块内容区分，放在同一个文件中。

## 自定义损失函数
在机器学期过程中，我们常常会使用损失函数（cost function)来优化模型，以满足实际需求。除了经典的损失函数外，有时候我们需要根据实际情况定义一些损失函数。

比如说，在预测商品价格时，预测多了，商家损失的是生产商品的生产成本；预测少了，损失的是商品的利润。考虑到商品的成本和利润不会严格相等，假如一个商品的成本是1元，但利润是10元，当我们使用如**均方误差**这样的损失函数时，有可能无法实现利润最大化。所以，此时我们需要定义一个损失函数，该损失函数需要与利润直接联系起来。

In [ ]:
# 定义损失函数如下：
"""
loss = tf.reduce_sum(tf.where(tf.greater(v1, v2),
                               (v1 - v2) * a, (v2 - v1) * b))
"""
# 当v1大于v2时，执行（v1 - v2) * a；当v1不大于v2时，执行（v2 - v1) * b


In [ ]:
# tf.where和tf.greater的实例
import tensorflow as tf
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])

sess = tf.InteractiveSession()
print tf.greater(v1, v2).eval()
# 输出[False False True True]

print tf.where(tf.greater(v1, v2), v1, v2).eval()
# 输出[4. 3. 3. 4.]

sess.close()

下面用一个简单的神经网络实现上面的这个自定义函数
* 两层的神经网络
* 两个输入节点，一个输出节点
* 数据集随机生成

In [ ]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 预测多了和预测少了的成本
loss_less = 10
loss_more = 1

loss = tf.reduce_sum(tf.where(tf.greater(y, y_),
                               (y - y_) * loss_more, 
                               (y_ - y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 随机生成数据集，噪声为-0.05 ～ 0.05的随机数
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[x1 + x2 + rdm.rand()/10.0 - 0.05] for (x1, x2) in X]

# 训练神经网络
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict = {x: X[start: end],
                                          y_: Y[start: end]})
        print sess.run(w1)

# 滑动平均模型（简单样例）
这是一种可以使得模型在测试数据上更健壮（robust）的方法，在采用随机梯度下降算法训练神经网络时，可以在一定程度上提升最终模型在测试数据上的表现。

使用的函数：
tf.train.ExponentialMovingAverage

每次使用的衰减率：
min{decay, (1+num_updates)/(10+num_updates)

每次更新变量的值：
shadow_variable = decay*shadow_variable + (1-decay)*variable

In [ ]:
import tensorflow as tf

v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均的类,衰减率为0.99, 控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)

# 定义一个更新滑动平均的操作
maintain_averages_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print sess.run([v1, ema.average(v1)])
    
    # 更新v1的值为5,更新结果为4.5
    sess.run(tf.assign(v1, 5))
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])
    
    # 更新step的值为10000
    sess.run(tf.assign(step, 10000))
    # 更新v1的值为10,更新结果为4.555
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])
    
    # 再次更新，结果为4.60945
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])

# 模型持久化
Tensorflow提供了tf.train.Saver类用来保存和还原一个神经网络模型

In [1]:
# 保存模型
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "./saver/model_eg.ckpt")

In [3]:
# 加载计算图，不用重复图中的计算流程
import tensorflow as tf

saver = tf.train.import_meta_graph("./saver/model_eg.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "./saver/model_eg.ckpt")
    print sess.run(tf.get_default_graph().get_tensor_by_name("add:0"))


INFO:tensorflow:Restoring parameters from ./saver/model_eg.ckpt
[ 3.]


In [6]:
# 加载时变量重命名
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='other-v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='other-v2')

saver = tf.train.Saver({"v1":v1, "v2":v2})
print v1, v2

<tf.Variable 'other-v1_1:0' shape=(1,) dtype=float32_ref> <tf.Variable 'other-v2_1:0' shape=(1,) dtype=float32_ref>


以上功能的主要目的之一是，为了方便使用变量的滑动平均值。以下是实现的代码：

In [10]:
import tensorflow as tf

v = tf.Variable(0, dtype=tf.float32, name="v")
for variables in tf.global_variables():
    print variables.name
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
for variables in tf.global_variables():
    print variables.name
    
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    
    saver.save(sess, "./saver/model.ckpt")
    print sess.run([v, ema.average(v)])

v1:0
v2:0
v1_1:0
v2_1:0
v1:0
v2:0
v1_3:0
v2_3:0
other-v1:0
other-v2:0
other-v1_1:0
other-v2_1:0
v:0
v1/ExponentialMovingAverage:0
v2/ExponentialMovingAverage:0
v1_1/ExponentialMovingAverage:0
v2_1/ExponentialMovingAverage:0
v_1:0
v1/ExponentialMovingAverage_1:0
v2/ExponentialMovingAverage_1:0
v1_1/ExponentialMovingAverage_1:0
v2_1/ExponentialMovingAverage_1:0
v_2:0
v1/ExponentialMovingAverage_2:0
v2/ExponentialMovingAverage_2:0
v1_1/ExponentialMovingAverage_2:0
v2_1/ExponentialMovingAverage_2:0
v_3:0


AttributeError: 'Variable' object has no attribute '_initial_value'

In [12]:
# variables_to_restore函数
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
print ema.variables_to_restore()

saver = tf.train.Saver({"v/ExpinentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "./saver/model.ckpt")
    print sess.run(v)

{u'v2_1/ExponentialMovingAverage_4': <tf.Variable 'v2_1:0' shape=(1,) dtype=float32_ref>, u'v2_1/ExponentialMovingAverage_1': <tf.Variable 'v2_1/ExponentialMovingAverage_1:0' shape=(1,) dtype=float32_ref>, u'v1_3/ExponentialMovingAverage': <tf.Variable 'v1_3:0' shape=(1,) dtype=float32_ref>, u'v2_1/ExponentialMovingAverage_3': <tf.Variable 'v2_1/ExponentialMovingAverage_3:0' shape=(1,) dtype=float32_ref>, u'v2_1/ExponentialMovingAverage_2': <tf.Variable 'v2_1/ExponentialMovingAverage_2:0' shape=(1,) dtype=float32_ref>, u'v2/ExponentialMovingAverage_3': <tf.Variable 'v2/ExponentialMovingAverage_3:0' shape=(1,) dtype=float32_ref>, u'v2/ExponentialMovingAverage_2': <tf.Variable 'v2/ExponentialMovingAverage_2:0' shape=(1,) dtype=float32_ref>, u'v2/ExponentialMovingAverage_1': <tf.Variable 'v2/ExponentialMovingAverage_1:0' shape=(1,) dtype=float32_ref>, u'v2/ExponentialMovingAverage_4': <tf.Variable 'v2:0' shape=(1,) dtype=float32_ref>, u'v/ExponentialMovingAverage': <tf.Variable 'v:0' shap

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on ./saver/model.ckpt: Not found: ./saver
	 [[Node: save_5/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_5/Const_0_0, save_5/RestoreV2/tensor_names, save_5/RestoreV2/shape_and_slices)]]

Caused by op u'save_5/RestoreV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/vmoonodd/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/vmoonodd/tensorflow/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/vmoonodd/tensorflow/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.ins